In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
from torch.optim import lr_scheduler

from torchvision import models,datasets
import torchvision.transforms as transforms

from PIL import Image

import os
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 数据强化部分

transform = transforms.Compose([
    transforms.ToPILImage(),
    
    transforms.Grayscale(1),#转成灰图
    transforms.RandomGrayscale(),#随机一部分转成灰图
    
    transforms.RandomOrder(),#随机顺序进行操作
    transforms.TandomApply(transforms,p=0.5),#随机选择进行操作
    transforms.RandomChoice(),#随即操作
    
    transforms.Pad(4),#padding
    transforms.RandomHorizontalFlip(0.5),#随机水平旋转
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomAffine(),#随机偏移
    transforms.RandomRotation(60),#旋转
    
    transforms.CenterCrop(),#中心剪裁
    transforms.FiveCrop(),#五角剪裁
    transforms.RandomCrop(28),#随机剪裁
    transforms.RandomResizedCrop(28),#旋转剪裁
    transforms.Resize(28),
    
    transforms,RandomErasing(0.5),
    
    transforms.ColorJitter(),#改变亮度
    transforms.RandomPerspective(),#改变透视度
    
    transforms.Normalize(mean=[0,0,0],std=[0,0,0])
    
    transforms.ToTensor()])


In [2]:
transform= transforms.Compose([
    transforms.ToPILImage(), 
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(60),
    transforms.RandomResizedCrop(28),
    transforms.ToTensor(),
])

## 数据封装部分

In [3]:
class trainDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.path="./train.csv"
        
        train_data=pd.read_csv(self.path)
        train_img= train_data.drop('label', axis=1)
        train_label = train_data[['label']]
        train_img=np.array(train_img).reshape(-1,1,28,28)
        train_img=torch.Tensor(train_img)
        train_img=train_img/255
        self.train_img=(train_img-train_img.mean())/train_img.std()
        
        train_label=np.array(train_label)
        train_label=train_label.reshape(-1)
        self.train_label=torch.LongTensor(train_label)
        
        
        self.len=self.train_img.shape[0]
        
    def __getitem__(self, index):
        # 2. Preprocess the data (e.g. torchvision.Transform).
        #self.train_img[index]=transform(self.train_img[index])
        # 3. Return a data pair (e.g. image and label).
        return (self.train_img[index],self.train_label[index])
    
    def __len__(self):
        return self.len


train_dataset=datasets.MNIST(root='./data/',
                          train=True,
                          transform=transform,
                          download=True)
test_dataset=datasets.MNIST(root='./data/',
                            train=False,
                            transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1000, 
                                           shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                          batch_size=64, 
#                                           shuffle=True)

In [4]:
class testDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.path="./test.csv"
        
        test_data=pd.read_csv(self.path)
        test_img=np.array(test_data).reshape(-1,1,28,28)
        test_img=torch.Tensor(test_img)/255
        self.test_img=(test_img-test_img.mean())/test_img.std()
        
        
        self.test_label=torch.LongTensor(self.test_img.shape[0])
        
        self.len=self.test_img.shape[0]
        
    def __getitem__(self, index):
        # 2. Preprocess the data (e.g. torchvision.Transform).
        #self.test_img[index]=transform(self.test_img[index])
        # 3. Return a data pair (e.g. image and label).
        return (self.test_img[index],self.test_label[index])
        
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return self.len


## 迁移学习部分

In [5]:
train_dataset = trainDataset()
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=500, 
                                           shuffle=True)
test_dataset = testDataset()
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=500, 
                                           shuffle=False)

In [12]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            #nn.BatchNorm2d(32),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(128, 192, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            #nn.BatchNorm2d(128),
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(192, 192, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),)
        
        self.flatten=nn.Flatten()
        
        self.fc1 =  nn.Sequential(
                        nn.Linear(192*3*3, 256),
                        nn.ReLU(),)
        #self.drop1=nn.Dropout(0.4)
        #self.fc2 =  nn.Sequential(
        #                nn.Linear(256, 256),
        #                nn.ReLU(),)
        #self.drop2=nn.Dropout(0.3)
        self.fc2 = nn.Linear(256,num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out =self.layer4(out)
        
        out=self.flatten(out)
        
        out = self.fc1(out)
        #out=self.drop1(out)
        out=self.fc2(out)
        #out-self.drop2(out)
        #out=self.fc3(out)
        return out

model = ConvNet(10).to(device)

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True      
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,10)
model.conv1=nn.Conv2d(1,64,7,2,3)
model.to(device)
print()

## 损失函数和优化器

In [13]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## 训练

In [15]:
total_epoch=50
model.train(True)

for epoch in range(total_epoch):
    for i, (images, labels) in enumerate(train_loader):
        #model.train()
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, total_epoch, i+1, len(train_dataset)//500, loss.item()))
            #model.eval()
            #with torch.no_grad():
            #    outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total=labels.shape[0]
            correct=(predicted==labels).sum()
            print('current Accuracy :{} %'.format(100*correct/total))
    #scheduler.step()
            

Epoch [1/50], Step [10/84], Loss: 0.0411
current Accuracy :98 %
Epoch [1/50], Step [20/84], Loss: 0.0561
current Accuracy :98 %
Epoch [1/50], Step [30/84], Loss: 0.0336
current Accuracy :99 %
Epoch [1/50], Step [40/84], Loss: 0.0404
current Accuracy :99 %
Epoch [1/50], Step [50/84], Loss: 0.0452
current Accuracy :98 %
Epoch [1/50], Step [60/84], Loss: 0.0345
current Accuracy :98 %
Epoch [1/50], Step [70/84], Loss: 0.0386
current Accuracy :98 %
Epoch [1/50], Step [80/84], Loss: 0.0307
current Accuracy :99 %
Epoch [2/50], Step [10/84], Loss: 0.0246
current Accuracy :99 %
Epoch [2/50], Step [20/84], Loss: 0.0166
current Accuracy :99 %
Epoch [2/50], Step [30/84], Loss: 0.0333
current Accuracy :98 %
Epoch [2/50], Step [40/84], Loss: 0.0258
current Accuracy :99 %
Epoch [2/50], Step [50/84], Loss: 0.0217
current Accuracy :99 %
Epoch [2/50], Step [60/84], Loss: 0.0220
current Accuracy :99 %
Epoch [2/50], Step [70/84], Loss: 0.0485
current Accuracy :98 %
Epoch [2/50], Step [80/84], Loss: 0.0284

torch.save(model.state_dict(), './mymodel.pkl')

model.load_state_dict(torch.load('./mymodel.pkl'))

## 评估

In [16]:
model.eval()
model=model.to(device)

predicted=torch.LongTensor(0)

for i, (images, labels) in enumerate(test_loader):
    with torch.no_grad():
        
        images=images.to(device)
        labels=labels.to(device)
        
        correct = 0
        total = 0
        outputs = model(images)
        _, predicted_tem = torch.max(outputs.data, 1)
        predicted=torch.cat((predicted,predicted_tem.cpu()),0)

In [17]:
test_label=predicted.cpu().numpy()
test_label = pd.Series(test_label,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),test_label],axis = 1)
submission.to_csv("submission.csv",index=False)